In [1]:
import org.apache.spark

Intitializing Scala interpreter ...

Spark Web UI available at http://10.201.100.218:4040
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1736429560828)
SparkSession available as 'spark'


import org.apache.spark


In [ ]:
// DO NOT EXECUTE - this is needed just to avoid showing errors in the following cells
val sc = spark.SparkContext.getOrCreate()

In [2]:
// all the imports
import java.io.{BufferedWriter, File, FileWriter}
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.module.scala.DefaultScalaModule
import scala.collection.mutable
import scala.io.Source

import java.io.{BufferedWriter, File, FileWriter}
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.module.scala.DefaultScalaModule
import scala.collection.mutable
import scala.io.Source


In [3]:
val path_to_datasets = "../../../datasets/spotify/data/"
val path_to_output = "../../../datasets/output/"

path_to_datasets: String = ../../../datasets/spotify/data/
path_to_output: String = ../../../datasets/output/


In [4]:
case class Track(uri: String, name: String, duration: Int, artistUri: String, albumUri: String, albumName: String)

case class Playlist(pid: Int, name: String, numFollowers: Int)

case class TrackInPlaylist(pid: Int, trackUri: String, pos: Int)

case class Artist(uri: String, name: String)

defined class Track
defined class Playlist
defined class TrackInPlaylist
defined class Artist


In [9]:
// write the output in csv
def appendToCsv[T](data: Seq[T], csvFile: String, headers: Seq[String])(implicit toMap: T => Map[String, Any]): Unit = {
  val file = new File(csvFile)
  val outputDir = file.getParentFile
    if (!outputDir.exists() && outputDir != null) {
        outputDir.mkdirs()
    }
  val writer = new BufferedWriter(new FileWriter(file, true))
  try {
    if(!file.exists() || file.length() == 0) {
      writer.write(headers.mkString(",") + "\n")
    }
    data.foreach { row =>
      val line = headers.map(h => toMap(row).getOrElse(h, "").toString).mkString(",")
      writer.write(line + "\n")
    }
  } finally {
    writer.close()
  }
}

appendToCsv: [T](data: Seq[T], csvFile: String, headers: Seq[String])(implicit toMap: T => Map[String,Any])Unit


In [6]:
// implicit conversion to Map
implicit val playlistToMap: Playlist => Map[String, Any] = p => Map(
    "pid" -> p.pid,
    "name" -> p.name,
    "numFollowers" -> p.numFollowers
)
implicit val trackToMap: Track => Map[String, Any] = t => Map(
  "uri" -> t.uri,
  "name" -> t.name,
  "duration" -> t.duration,
  "artistUri" -> t.artistUri,
  "albumUri" -> t.albumUri,
  "albumName" -> t.albumName
)
implicit val trackInPlaylistToMap: TrackInPlaylist => Map[String, Any] = t => Map(
  "pid" -> t.pid,
  "trackUri" -> t.trackUri,
  "pos" -> t.pos
)
implicit val artistToMap: Artist => Map[String, Any] = a => Map(
  "uri" -> a.uri,
  "name" -> a.name
)


playlistToMap: Playlist => Map[String,Any] = $Lambda$2428/0x0000000800f10840@57a44979
trackToMap: Track => Map[String,Any] = $Lambda$2429/0x0000000800f11040@591efccc
trackInPlaylistToMap: TrackInPlaylist => Map[String,Any] = $Lambda$2430/0x0000000800f12040@530b08af
artistToMap: Artist => Map[String,Any] = $Lambda$2431/0x0000000800f12840@37ac5f31


In [11]:
val mapper = new ObjectMapper()
mapper.registerModule(DefaultScalaModule)
val files = new File(path_to_datasets).listFiles().filter(_.getName.endsWith(".json"))

// path to the output files
val playlistsCsv = path_to_output + "playlists.csv"
val tracksCsv = path_to_output + "tracks.csv"
val trackInPlaylistCsv = path_to_output + "track_in_playlist.csv"
val artistsCsv = path_to_output + "artists.csv"

var counter = 0
// prendo i primi 200 file
for (file <- files.take(10)) {
  println(s"Processing file: ${counter}")
  //increment the counter
  counter += 1
  
  val jsonString = Source.fromFile(file).getLines().mkString
  val jsonData = mapper.readValue(jsonString, classOf[Map[String, Any]])
  val playlistsRaw = jsonData("playlists").asInstanceOf[List[Map[String, Any]]]

  // list for the data
  val playlists = mutable.ListBuffer[Playlist]()
  val tracks = mutable.ListBuffer[Track]()
  val trackInPlaylist = mutable.ListBuffer[TrackInPlaylist]()
  val artists = mutable.Set[Artist]()
  
  // Converte i dati delle playlist
  playlistsRaw.foreach { playlist =>
    playlists += Playlist(
      pid = playlist("pid").asInstanceOf[Int],
      name = playlist("name").toString,
      numFollowers = playlist("num_followers").asInstanceOf[Int]
    )
    
    // Converte i dati dei brani
    val tracksInPlaylist = playlist("tracks").asInstanceOf[List[Map[String, Any]]]
    tracksInPlaylist.zipWithIndex.foreach { case (track, index) =>
      tracks += Track(
        uri = track("track_uri").toString,
        name = track.getOrElse("track_name", "").toString,
        duration = track.getOrElse("duration_ms", 0).asInstanceOf[Int],
        artistUri = track("artist_uri").toString,
        albumUri = track.getOrElse("album_uri", "").toString,
        albumName = track.getOrElse("album_name", "").toString
      )
      
      trackInPlaylist += TrackInPlaylist(
        pid = playlist("pid").asInstanceOf[Int],
        trackUri = track("track_uri").toString,
        pos = index
      )

      artists += Artist(
        uri = track("artist_uri").toString,
        name = track("artist_name").toString // Nome artista non presente nel JSON
      )
    }
  }
  // write in csv
  appendToCsv(playlists.toSeq, playlistsCsv, Seq("pid", "name", "numFollowers"))
  appendToCsv(tracks.toSeq, tracksCsv, Seq("uri", "name", "duration", "artistUri", "albumUri", "albumName"))
  appendToCsv(trackInPlaylist.toSeq, trackInPlaylistCsv, Seq("pid", "trackUri", "pos"))
  appendToCsv(artists.toSeq, artistsCsv, Seq("uri", "name"))  
}


Processing file: 0
Processing file: 1
Processing file: 2
Processing file: 3
Processing file: 4
Processing file: 5
Processing file: 6
Processing file: 7
Processing file: 8
Processing file: 9


mapper: com.fasterxml.jackson.databind.ObjectMapper = com.fasterxml.jackson.databind.ObjectMapper@6028c9e4
files: Array[java.io.File] = Array(..\..\..\datasets\spotify\data\mpd.slice.0-999.json, ..\..\..\datasets\spotify\data\mpd.slice.1000-1999.json, ..\..\..\datasets\spotify\data\mpd.slice.10000-10999.json, ..\..\..\datasets\spotify\data\mpd.slice.100000-100999.json, ..\..\..\datasets\spotify\data\mpd.slice.101000-101999.json, ..\..\..\datasets\spotify\data\mpd.slice.102000-102999.json, ..\..\..\datasets\spotify\data\mpd.slice.103000-103999.json, ..\..\..\datasets\spotify\data\mpd.slice.104000-104999.json, ..\..\..\datasets\spotify\data\mpd.slice.105000-105999.json, ..\..\..\datasets\spotify\data\mpd.slice.106000-106999.json, ..\..\..\datasets\spotify\data\mpd.slice.107000-107999.jso...


In [ ]:
// read the csv files and delete the duplicate, with spark
